In [2]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')
df = pd.read_csv("/content/drive/MyDrive/3A2M.csv")

Mounted at /content/drive


In [3]:
!pip install transformers torch pandas scikit-learn

import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import GPT2Tokenizer

# Load your CSV (downloaded from Kaggle link)
df = pd.read_csv("/content/drive/MyDrive/3A2M.csv")   # change file name if needed
display(df.head())

,Unnamed: 0,title,directions,NER,genre,label
0,0,Reeses Cups(Candy),"[""Combine first four ingredients and press in ...","[""peanut butter"", ""graham cracker crumbs"", ""bu...",drinks,2
1,1,Rhubarb Coffee Cake,"[""Cream sugar and butter."", ""Add egg and beat ...","[""sugar"", ""butter"", ""egg"", ""buttermilk"", ""flou...",drinks,2
2,2,Quick Barbecue Wings,"[""Clean wings."", ""Flour and fry until done."", ...","[""chicken"", ""flour"", ""barbecue sauce""]",nonveg,3
3,3,Chocolate Frango Mints,"[""Mix ingredients together for 5 minutes."", ""S...","[""cake mix"", ""chocolate fudge pudding"", ""sour ...",drinks,2
4,4,Corral Barbecued Beef Steak Strips,"[""Brown strips in cooking oil."", ""Pour off dri...","[""long"", ""cooking oil"", ""tomato sauce"", ""water...",drinks,2


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# 1. Load your dataset
CSV_PATH = "/content/drive/MyDrive/3A2M.csv"  # change path if needed
df = pd.read_csv(CSV_PATH)

# 2. Combine text fields safely
df["text"] = (
    "Recipe: " + df.get("title", df.get("name", "Unknown")) +
    "\nIngredients: " + df.get("NER", df.get("ingredients", "")) +
    "\nInstructions: " + df.get("directions", df.get("steps", ""))
)

# 3. Split data
train_texts, val_texts = train_test_split(df["text"].dropna(), test_size=0.2, random_state=42)

# 4. Fast batch tokenization
tok = GPT2Tokenizer.from_pretrained("gpt2")
tok.pad_token = tok.eos_token

def fast_tokenize(texts, batch_size=512):
    encs = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i+batch_size]
        enc = tok(batch, truncation=True, padding="max_length", max_length=256)
        encs.append(enc)
    return {
        "input_ids": sum([e["input_ids"] for e in encs], []),
        "attention_mask": sum([e["attention_mask"] for e in encs], [])
    }

train_enc = fast_tokenize(list(train_texts))
val_enc = fast_tokenize(list(val_texts))

print("Data ready!")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

 81%|████████▏ | 2834/3487 [25:22<07:15,  1.50it/s]

In [1]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.eval()

prompt = "Recipe: Pancakes\nIngredients: flour, milk, egg, sugar\nInstructions:"
inputs = tokenizer(prompt, return_tensors="pt")

output = model.generate(**inputs, max_length=150, temperature=0.8, top_p=0.9, do_sample=True, pad_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(output[0], skip_special_tokens=True))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Recipe: Pancakes
Ingredients: flour, milk, egg, sugar
Instructions: 1. Preheat oven to 375°F. Line a baking sheet with parchment paper and cut out the outer crust. Using a fork or spatula, roll out the outer crust about a 1/4-inch thick. Using a small spatula, roll out the inner crust about 1/4-inch thick. Using a large spatula, roll out the outer crust about 1/4-inch thick. Using a small spatula, roll out the outer crust about 2-inches thick. Using a large spatula, roll out the outer crust about 1/4-inch thick. Place the pancake halves on the prepared baking sheet and bake for


In [2]:
from google.colab import drive
drive.mount('/content/drive')

model.save_pretrained("/content/drive/MyDrive/gpt2_recipe_model")
tokenizer.save_pretrained("/content/drive/MyDrive/gpt2_recipe_model")
print(" Model permanently saved in Drive at /MyDrive/gpt2_recipe_model")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 Model permanently saved in Drive at /MyDrive/gpt2_recipe_model


In [3]:
from huggingface_hub import login

# When prompted, paste your new token (do NOT store in notebook)
login()


In [4]:
from huggingface_hub import whoami
print(whoami())


{'type': 'user', 'id': '67d83e1ac694afe74b4f32e3', 'name': 'MuqadusAmsha', 'fullname': 'Amsha', 'email': 'f247812@cfd.nu.edu.pk', 'emailVerified': True, 'canPay': False, 'periodEnd': None, 'isPro': False, 'avatarUrl': '/avatars/e4d19a806b185c30abfc26c25e500611.svg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'recipe-generator-app', 'role': 'write', 'createdAt': '2025-10-26T10:44:40.061Z'}}}


In [17]:
from huggingface_hub import upload_folder

upload_folder(
    folder_path="/content/drive/MyDrive/gpt2_recipe_model",  # your saved model path
    repo_id="MuqadusAmsha/gpt2-recipe-maker",
    repo_type="model",
)


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...e_model/model.safetensors:   5%|5         | 25.1MB /  498MB            

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/MuqadusAmsha/gpt2-recipe-maker/commit/0efdf74d3de3920b924f7c43cda708fa90da006c', commit_message='Upload folder using huggingface_hub', commit_description='', oid='0efdf74d3de3920b924f7c43cda708fa90da006c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/MuqadusAmsha/gpt2-recipe-maker', endpoint='https://huggingface.co', repo_type='model', repo_id='MuqadusAmsha/gpt2-recipe-maker'), pr_revision=None, pr_num=None)

In [26]:
!ls /content


drive  requirements.txt  sample_data


In [28]:
from huggingface_hub import login, upload_folder

# 1 Log in with your Hugging Face token (if not already logged in)
login()

# 2 Upload your model from Google Drive to the existing repo
upload_folder(
    folder_path="/content/drive/MyDrive/gpt2_recipe_model",  #  Correct Drive path
    repo_id="MuqadusAmsha/recipe-generator-app"
)


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...e_model/model.safetensors:   5%|5         | 25.1MB /  498MB            

CommitInfo(commit_url='https://huggingface.co/MuqadusAmsha/recipe-generator-app/commit/35890a60fcf45185a445982a35d9c7299deb1d13', commit_message='Upload folder using huggingface_hub', commit_description='', oid='35890a60fcf45185a445982a35d9c7299deb1d13', pr_url=None, repo_url=RepoUrl('https://huggingface.co/MuqadusAmsha/recipe-generator-app', endpoint='https://huggingface.co', repo_type='model', repo_id='MuqadusAmsha/recipe-generator-app'), pr_revision=None, pr_num=None)

In [38]:
%%writefile app.py
import streamlit as st
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load model
MODEL_PATH = "MuqadusAmsha/gpt2-recipe-generator"

@st.cache_resource
def load_model():
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    model = AutoModelForCausalLM.from_pretrained(MODEL_PATH)
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.eos_token_id
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    return tokenizer, model, device

tokenizer, model, device = load_model()

def clean_text(text):
    lines = [l.strip() for l in text.split("\n") if l.strip()]
    cleaned, seen = [], set()
    for line in lines:
        low = line.lower()
        if low not in seen and len(line.split()) > 2:
            if not line.endswith(":"):
                cleaned.append(line)
                seen.add(low)
    return "\n".join(cleaned)

st.title("Recipe Generator")
st.write("Type a dish name or list of ingredients to generate a clear and realistic recipe.")

text = st.text_area("Enter dish or ingredients:")

if st.button("Generate Recipe"):
    if not text.strip():
        st.warning("Please enter something first.")
    else:
        prompt = (
            f"Write a clear, realistic cooking recipe for {text}. "
            f"Include:\nIngredients list and numbered Instructions. Avoid repetition."
        )

        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=200,
                temperature=0.8,
                top_p=0.9,
                repetition_penalty=3.0,
                no_repeat_ngram_size=5,
                do_sample=True,
                early_stopping=True,
                pad_token_id=tokenizer.eos_token_id
            )

        recipe = tokenizer.decode(outputs[0], skip_special_tokens=True)
        recipe = recipe.replace(prompt, "").strip()
        recipe = clean_text(recipe)
        recipe = recipe.replace("Ingredients", "\n\nIngredients").replace("Instructions", "\n\nInstructions")

        st.subheader("Generated Recipe:")
        st.text(recipe)



Overwriting app.py


In [30]:
pip install transformers torch streamlit


In [37]:
%%writefile requirements.txt
torch
transformers
streamlit
huggingface-hub



Overwriting requirements.txt


In [32]:
!git add app.py requirements.txt
!git commit -m "Fix dependency versions for Streamlit Cloud"
!git push origin main


fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git


In [33]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch, re

model_path = "MuqadusAmsha/gpt2-recipe-generator"
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

prompt = "Chocolate cake with cream cheese frosting"
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Generate more focused recipe text
outputs = model.generate(
    **inputs,
    max_length=160,
    temperature=0.7,        # lower temperature = more factual
    top_p=0.85,             # slightly tighter sampling
    repetition_penalty=2.0, # avoid repetitive words
    no_repeat_ngram_size=3,
    pad_token_id=tokenizer.eos_token_id,
    do_sample=True
)

# Decode and clean
text = tokenizer.decode(outputs[0], skip_special_tokens=True)
text = re.sub(r"\s+", " ", text)
text = text.replace("Recipe:", "\n\nRecipe:").replace("Ingredients:", "\n\nIngredients:").replace("Instructions:", "\n\nInstructions:")

# Keep only the first part (avoid long continuations)
cut_points = ["Instructions:", "Directions:", "Method:"]
for cut in cut_points:
    if cut in text:
        text = text.split(cut)[0] + cut + text.split(cut)[1].split(".")[0] + "."

print("\nGenerated Recipe:\n")
print(text)



Generated Recipe:

Chocolate cake with cream cheese frosting And I've been using this for a few years now. The only thing that keeps it from being too heavy is the fact of how much chocolate you can put on top! It's soooo easy to make, and even easier than baking in your oven or microwave (I used my own mini-frosted cookie sheet). There are many different types available at Whole Foods because they have all their ingredients listed above: cookies; candy bars/flavors such as raisins & mints); frozen candies like graham crackers which also come pre packed into one box ; etc... And there will always be some extra stuff inside :) In any case though if anyone has suggestions please share them here :D
